In [112]:
%load_ext autoreload
%autoreload

import os
import sys
import random
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from datetime import datetime
import time
from collections import Counter
import pandas as pd
import numpy as np
import random
from tqdm import tqdm_notebook

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
bgr = m.BalancegameRecord.objects.filter(who=694).exclude(tag_pos__isnull=True).select_related('tag_pos', 'tag_neg').order_by('-id')[:100]

flds = [
    'type','category','item',
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

df_pos = pd.DataFrame([r.tag_pos.__dict__ for r in bgr])[flds]
df_neg = pd.DataFrame([r.tag_neg.__dict__ for r in bgr])[flds]

In [135]:
def get_baseframe(df0):
    _df = df0.set_index(['type','category','item']).stack().to_frame()
    _df[1] = 1
    _df.index.names = ['type','category','item','prop']
    _df.columns = ['val', 'count']
    return _df.groupby(by=['type','category','item','prop','val']).count()

In [136]:
df_pos = get_baseframe(df_pos)
df_neg = get_baseframe(df_neg)

In [140]:
# df = df_pos - df_neg
df = df_pos.sub(df_neg, fill_value=0)
df = df[df['count'] > 0]
df#['count'].sum()

count
type        category item prop      val         
accessories 잡화       헤어밴드 color     핑크       1.0
                          detail    리본       1.0
                          pattern   무지       1.0
bags        가방       토트백  color     그린       1.0
                          detail    지퍼       1.0
...                                          ...
shoes       부츠/워커    워커   pattern   무지       1.0
                          sole_type 플랫폼      1.0
                          strap     False    1.0
                          texture   가죽       1.0
                          toe_type  라운드토     1.0

[81 rows x 1 columns]

In [109]:
df.reset_index().to_dict('records')

[{'type': 'accessories',
  'category': '잡화',
  'item': '손목시계',
  'prop': 'texture',
  'val': '가죽',
  'count': 1.0},
 {'type': 'clothing',
  'category': '니트웨어',
  'item': '가디건',
  'prop': 'neckline',
  'val': '노카라',
  'count': 1.0},
 {'type': 'clothing',
  'category': '니트웨어',
  'item': '가디건',
  'prop': 'texture',
  'val': '니트',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'fit',
  'val': '노멀',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'length',
  'val': '노멀',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'look',
  'val': '캐주얼',
  'count': 2.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'neckline',
  'val': '브이넥',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'pattern',
  'val': '무지',
  'count': 1.0},
 {'type': 'clothing',
  'category': '베스트',
  'item': '베스트',
  'prop': 'sleeve_length',
  'val'

In [27]:
df['count'].unstack()

val                                        가죽  노카라   니트   노멀  캐주얼  브이넥   무지  \
type        category item  sub                                                
accessories 잡화       손목시계  texture        1.0  NaN  NaN  NaN  NaN  NaN  NaN   
clothing    니트웨어     가디건   texture        NaN  NaN  1.0  NaN  NaN  NaN  NaN   
                           neckline       NaN  1.0  NaN  NaN  NaN  NaN  NaN   
            베스트      베스트   texture        NaN  NaN  2.0  NaN  NaN  NaN  NaN   
                           neckline       NaN  NaN  NaN  NaN  NaN  1.0  NaN   
                           fit            NaN  NaN  NaN  2.0  NaN  NaN  NaN   
                           length         NaN  NaN  NaN  2.0  NaN  NaN  NaN   
                           look           NaN  NaN  NaN  NaN  2.0  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  1.0   
                           sleeve_length  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           strap          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            블라우스     블라우스  fit            NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           length         NaN  NaN  NaN  1.0  NaN  NaN  NaN   
                           look           NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  2.0   
                           color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            셔츠       셔츠    texture        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           fit            NaN  NaN  NaN  1.0  NaN  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  2.0   
                           color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           detail         NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            점퍼       블루종   texture        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           length         NaN  NaN  NaN  1.0  NaN  NaN  NaN   
                           look           NaN  NaN  NaN  NaN  1.0  NaN  NaN   
                           sleeve_length  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           strap          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            청바지      청바지   texture        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           fit            NaN  NaN  NaN  3.0  NaN  NaN  NaN   
                           look           NaN  NaN  NaN  NaN  1.0  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  1.0   
                           strap          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
hats        모자       캡     texture        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           strap          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           detail         NaN  NaN  NaN  NaN  NaN  NaN  NaN   
shoes       로퍼       로퍼    color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           detail         NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            운동화      스니커즈  texture        2.0  NaN  NaN  NaN  NaN  NaN  NaN   
                           pattern        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           strap          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           color          NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           detail         NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           sole_type      NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           toe_type       NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            정장구두     옥스포드화 heel_height    NaN  NaN  NaN  NaN  NaN  NaN  NaN   
            펌프스      펌프스   texture        NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                           color          NaN  NaN

In [72]:
maxsub = df['count'].unstack().idxmax(axis=1).to_frame()
maxsub

0
type        category item  sub                 
accessories 잡화       손목시계  texture           가죽
clothing    니트웨어     가디건   texture           니트
                           neckline         노카라
            베스트      베스트   texture           니트
                           neckline         브이넥
                           fit               노멀
                           length            노멀
                           look             캐주얼
                           pattern           무지
                           sleeve_length    민소매
                           strap          False
            블라우스     블라우스  fit            오버사이즈
                           length            노멀
                           look             페미닌
                           pattern           무지
                           color            화이트
            셔츠       셔츠    texture            면
                           fit               노멀
                           pattern           무지
                           color            화이트
                           detail            단추
            점퍼       블루종   texture         합성섬유
                           length            노멀
                           look             캐주얼
                           sleeve_length     긴팔
                           strap          False
                           color             블랙
            청바지      청바지   texture           데님
                           fit               노멀
                           look             캐주얼
                           pattern           무지
                           strap          False
hats        모자       캡     texture            면
                           pattern          레터링
                           strap          False
                           color             블랙
                           detail            자수
shoes       로퍼       로퍼    color             블랙
                           detail          레이스업
            운동화      스니커즈  texture           가죽
                           pattern           로고
                           strap          False
                           color            화이트
                           detail          레이스업
                           sole_type        플랫폼
                           toe_type        라운드토
            정장구두     옥스포드화 heel_height       플랫
            펌프스      펌프스   texture         스웨이드
                           color             블랙
                           toe_type        포인티토
                           heel_height       플랫
                           heel_shape      두꺼운굽

In [29]:
maxsub.loc['shoes']

0
category item  sub               
로퍼       로퍼    color           블랙
               detail        레이스업
운동화      스니커즈  texture         가죽
               pattern         로고
               strap        False
               color          화이트
               detail        레이스업
               sole_type      플랫폼
               toe_type      라운드토
정장구두     옥스포드화 heel_height     플랫
펌프스      펌프스   texture       스웨이드
               color           블랙
               toe_type      포인티토
               heel_height     플랫
               heel_shape    두꺼운굽

In [2]:
%%time
bgr = m.BalancegameRecord.objects.exclude(tag_pos__isnull=True).select_related('tag_pos', 'tag_neg')

df_pos = pd.DataFrame([r.tag_pos.__dict__ for r in bgr])
df_neg = pd.DataFrame([r.tag_neg.__dict__ for r in bgr])

flds = [
    'type','category','item',
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

count = {col:Counter(df_pos[col])-Counter(df_neg[col]) for col in flds }; count

CPU times: user 51.2 ms, sys: 4.36 ms, total: 55.5 ms
Wall time: 2.09 s


{'type': Counter(),
 'category': Counter({'운동화': 7, '목걸이': 2}),
 'item': Counter({'컨버스': 1,
          '청바지': 2,
          '슬리퍼': 1,
          '트렌치코트': 1,
          '버킷햇': 1,
          '모피코트': 2,
          '야상': 5,
          '에코백': 2,
          '런닝화': 5,
          '무스탕': 3,
          '스커트': 3,
          '팔찌/발찌': 2,
          '재킷': 7,
          '후드': 5,
          '탱크탑': 1,
          '스웨트팬츠': 5,
          '스웨터': 5,
          '목걸이': 2,
          '토트백': 2,
          '베스트': 3,
          '플리플랍': 2,
          '플리츠스커트': 3,
          '슬립드레스': 1,
          '뷔스티에': 2,
          '클러치': 1,
          '니트베스트': 1,
          '더플코트': 1,
          '카고팬츠': 1,
          '하이탑스니커즈': 1,
          '데님재킷': 3,
          '튜닉원피스': 2,
          '배기팬츠': 1,
          '바람막이': 6,
          '캡': 1,
          '와이드팬츠': 3,
          '벨트백': 1,
          '터틀넥': 2,
          '펌프스': 2,
          '니트원피스': 2,
          '홀터넥드레스': 3,
          '원피스수영복': 1,
          '워커': 2,
          '패딩조끼': 2,
          '와이셔츠': 1,
          '멜빵원피

In [3]:
dfdf_pos = df_pos[flds].set_index(['type','category','item']).stack().to_frame()
dfdf_pos[1] = 1
dfdf_pos.index.names = ['type','category','item','sub']
dfdf_pos.columns = ['val', 'count']
dfdf_pos = dfdf_pos.groupby(by=['type','category','item','sub','val']).count()

In [4]:
dfdf_neg = df_neg[flds].set_index(['type','category','item']).stack().to_frame()
dfdf_neg[1] = 1
dfdf_neg.index.names = ['type','category','item','sub']
dfdf_neg.columns = ['val', 'count']
dfdf_neg = dfdf_neg.groupby(by=['type','category','item','sub','val']).count()

In [5]:
dfdf = dfdf_pos - dfdf_neg
dfdf = dfdf[dfdf['count']>0]
dfdf

count
type        category item sub      val         
accessories 잡화       벨트   detail   버클       1.0
                     선글라스 color    블랙       2.0
                     손목시계 color    블랙       2.0
                          pattern  무지       1.0
                          texture  메탈       1.0
...                                         ...
shoes       펌프스      펌프스  strap    False    2.0
                          texture  가죽       1.0
                                   스웨이드     1.0
                          toe_type 오픈토      3.0
                                   포인티토     4.0

[430 rows x 1 columns]

In [6]:
dfdf['count'].unstack()

val                                     버클   블랙   무지   메탈  보우/타이  앱스트랙트  \
type        category item sub                                             
accessories 잡화       벨트   detail       1.0  NaN  NaN  NaN    NaN    NaN   
                     선글라스 color        NaN  2.0  NaN  NaN    NaN    NaN   
                     손목시계 color        NaN  2.0  NaN  NaN    NaN    NaN   
                          pattern      NaN  NaN  1.0  NaN    NaN    NaN   
                          texture      NaN  NaN  NaN  1.0    NaN    NaN   
...                                    ...  ...  ...  ...    ...    ...   
shoes       펌프스      펌프스  strap        NaN  NaN  NaN  NaN    NaN    NaN   
                          heel_height  NaN  NaN  NaN  NaN    NaN    NaN   
                          toe_type     NaN  NaN  NaN  NaN    NaN    NaN   
                          heel_shape   NaN  NaN  NaN  NaN    NaN    NaN   
                          sole_type    NaN  NaN  NaN  NaN    NaN    NaN   

val                                    플라스틱/PVC  브라운  화이트  밴딩  ...  원형굽  플랫폼  \
type        category item sub                                  ...             
accessories 잡화       벨트   detail            NaN  NaN  NaN NaN  ...  NaN  NaN   
                     선글라스 color             NaN  NaN  NaN NaN  ...  NaN  NaN   
                     손목시계 color             NaN  NaN  NaN NaN  ...  NaN  NaN   
                          pattern           NaN  NaN  NaN NaN  ...  NaN  NaN   
                          texture           NaN  NaN  NaN NaN  ...  NaN  NaN   
...                                         ...  ...  ...  ..  ...  ...  ...   
shoes       펌프스      펌프스  strap             NaN  NaN  NaN NaN  ...  NaN  NaN   
                          heel_height       NaN  NaN  NaN NaN  ...  NaN  NaN   
                          toe_type          NaN  NaN  NaN NaN  ...  NaN  NaN   
                          heel_shape        NaN  NaN  NaN NaN  ...  NaN  NaN   
                          sole_type         NaN  NaN  NaN NaN  ...  NaN  1.0   

val                                    스틸레토굽  앵클스트랩  고무  오픈토  아몬드토  하이힐   웨지  \
type        category item sub                                                  
accessories 잡화       벨트   detail         NaN    NaN NaN  NaN   NaN  NaN  NaN   
                     선글라스 color          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                     손목시계 color          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          pattern        NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          texture        NaN    NaN NaN  NaN   NaN  NaN  NaN   
...                                      ...    ...  ..  ...   ...  ...  ...   
shoes       펌프스      펌프스  strap          NaN    NaN NaN  NaN   NaN  NaN  NaN   
                          heel_height    NaN    NaN NaN  NaN   NaN  2.0  NaN   
                          toe_type       NaN    NaN NaN  3.0   NaN  NaN  NaN   
                          heel_shape     3.0    NaN NaN  NaN   NaN  NaN  NaN   
                          sole_type      NaN    NaN NaN  NaN   NaN  NaN  1.0   

val                                    포인티토  
type        category item sub                
accessories 잡화       벨트   detail        NaN  
                     선글라스 color         NaN  
                     손목시계 color         NaN  
                          pattern       NaN  
                          texture       NaN  
...                                     ...  
shoes       펌프스      펌프스  strap         NaN  
                          heel_height   NaN  
                          toe_type      4.0  
                          heel_shape    NaN  
                          sole_type     NaN  

[371 rows x 97 columns]

In [7]:
maxsub = dfdf['count'].unstack().idxmax(axis=1).to_frame()
maxsub

0
type        category item sub               
accessories 잡화       벨트   detail          버클
                     선글라스 color           블랙
                     손목시계 color           블랙
                          pattern         무지
                          texture         메탈
...                                      ...
shoes       펌프스      펌프스  strap        False
                          heel_height    하이힐
                          toe_type      포인티토
                          heel_shape   스틸레토굽
                          sole_type      플랫폼

[371 rows x 1 columns]

In [8]:
maxsub.loc['shoes']

0
category item sub               
로퍼       로퍼   detail          버클
              color          브라운
              texture       스웨이드
              heel_height    로우힐
              toe_type      라운드토
...                          ...
펌프스      펌프스  strap        False
              heel_height    하이힐
              toe_type      포인티토
              heel_shape   스틸레토굽
              sole_type      플랫폼

[81 rows x 1 columns]

In [ ]:
pd.pivot_table(dfdf, values='count', index=['type','category','item','sub'], columns=['val'], aggfunc='count')

In [ ]:
df_pos.groupby(by=['type','category','item','color']).count()

In [ ]:
flds = [
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

In [ ]:
%%time
out = {}
target_fld = 'color'
groupby_flds = ['type','category','item'] + [target_fld]
value_flds = groupby_flds + ['id']

_df = df_pos[value_flds].groupby(by=groupby_flds).count() - df_neg[value_flds].groupby(by=groupby_flds).count()
_df = _df.dropna()
_df[_df.id>0]

In [ ]:
%%time
_df1 = pd.pivot_table(df_pos, values='id', index=['type','category','item'], columns='color', aggfunc='count')
_df2 = pd.pivot_table(df_neg, values='id', index=['type','category','item'], columns='color', aggfunc='count')
_df1 - _df2

In [ ]:
sum(count.values(), Counter())